In [1]:
batch_size = 12
import torch
state_size  = 32
action_size = 5
import random
import numpy as np
from collections import deque,namedtuple
from environment import Env
import rospy
from sensor_msgs.msg import LaserScan
rospy.init_node("scan_test")
action_size = 5

env = Env(action_size)

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'done'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)


replay_buffer = ReplayMemory(capacity=10000000)




In [2]:
state = env.reset()

[INFO] [1683219744.187409, 0.888000]:  Goal Position : 0.6, 0.0


In [3]:
state

array([ 1.93204606,  1.97845924,  2.2540195 ,  0.75904226,  2.09119844,
        1.88717592,  1.84170556,  1.9729532 ,  0.68617195,  0.69372785,
        1.96063888,  1.79650843,  1.80898154,  1.95049703,  0.71345842,
        0.66947562,  1.95643473,  1.84448111,  1.88678002,  2.07985377,
        0.77345312,  2.2361784 ,  1.98881996,  1.91454971, -0.14      ,
        0.6       ,  0.67      , 15.        ])

In [11]:
env.step(3)

(array([ 0.14362858,  0.15363628,  1.47687078,  1.55032611,  1.75200498,
         2.19735432,  2.55710125,  0.91049135,  2.28158665,  2.43115473,
         2.79512763,  2.83655453,  2.41020346,  2.21352601,  0.83420712,
         2.4249661 ,  2.20188332,  1.71889484,  1.48298919,  1.41960657,
         1.44786334,  0.1805744 ,  0.13928185,  0.14292543,  1.74      ,
         0.5       ,  0.14      , 22.        ]),
 -3.1894174109424145,
 False)

In [3]:
for i in range(250):
    action = random.randrange(0,5)
    next_state, reward, done = env.step(action)
    replay_buffer.push(state, action, next_state, reward, done)
    state = next_state
    if done:
        state = env.reset()

[INFO] [1683219760.346820, 17.009000]: Goal!!
[INFO] [1683219761.251703, 17.892000]:  Goal Position : -1.7, -0.7
[INFO] [1683219774.211078, 30.814000]: Collision!!
[INFO] [1683219780.230980, 6.003000]: Collision!!
[INFO] [1683219787.461231, 7.207000]: Collision!!
[INFO] [1683219801.907819, 14.404000]: Collision!!


In [4]:
replay_buffer.memory[-1]

Transition(state=array([ 1.08942938,  1.03665102,  1.07575405,  1.18499732,  1.44869292,
        2.10414124,  1.98681605,  1.91683137,  0.55236691,  2.28772879,
        2.90325069,  1.39883065,  2.71119523,  2.68805909,  1.36656415,
        2.96159625,  0.52825278,  0.42218718,  1.79146302,  1.80656457,
        1.95087767,  1.58198738,  1.25725651,  1.0895102 , -1.63      ,
        1.08      ,  0.42      , 17.        ]), action=1, next_state=array([ 1.09154296,  1.02590036,  1.06856036,  1.16399765,  1.41220772,
        1.99238729,  1.99172187,  1.92121565,  0.57628804,  2.25355053,
        2.83385134,  1.43259847,  2.72524905,  2.69049215,  1.36524701,
        3.05577421,  2.27591896,  0.42024422,  1.80838382,  1.79396939,
        1.93985689,  1.62836194,  1.27964938,  1.08829546,  2.96      ,
        2.4       ,  0.42      , 17.        ]), reward=200, done=False)

In [5]:
import pickle
# save the replay buffer to disk

with open('replay_buffer.pkl', 'wb') as f:
    pickle.dump(replay_buffer, f)

In [5]:
replay_buffer.memory[-1]

IndexError: deque index out of range

In [6]:
# use the replay buffer from disk

with open('replay_buffer.pkl', 'rb') as f:
    replay_buffer = pickle.load(f)

In [7]:
len(replay_buffer)

250

In [18]:
sample = replay_buffer.sample(12)

In [19]:
sample = Transition(*zip(*sample))

In [20]:
batch = sample

In [21]:
state_batch = torch.tensor(np.array(batch.state), dtype=torch.float32)
action_batch = torch.tensor(np.array(batch.action), dtype=torch.int64)
reward_batch = torch.tensor(np.array(batch.reward), dtype=torch.float32)
next_batch = torch.tensor(np.array(batch.next_state), dtype=torch.float32)
done_batch = torch.tensor(batch.done, dtype =torch.bool)

In [16]:
state_batch.shape

torch.Size([12, 28])

In [18]:
action_batch

tensor([3, 4, 2, 1, 0, 4, 2, 0, 1, 0, 0, 4])

In [19]:
next_batch.shape

torch.Size([12, 28])

In [39]:
done_batch = torch.tensor([False, False, False, True, False, False, False, False, False, False,
        True, False])

In [22]:
from rl_model import DQNModel

In [23]:
policy_net = DQNModel(28, 5)

In [27]:
state_batch.dtype

torch.float64

In [24]:
o  = policy_net(state_batch).gather(1, action_batch.unsqueeze(1))

In [25]:
t = policy_net(next_batch).max(1)[0].detach()

In [37]:
t

tensor([0.3050, 0.4931, 0.1238, 0.8011, 0.4153, 0.6232, 0.4131, 0.1803, 0.5251,
        0.7435, 0.4612, 0.1322])

In [26]:
t[done_batch] = 0.0

In [41]:
t

tensor([0.3050, 0.4931, 0.1238, 0.0000, 0.4153, 0.6232, 0.4131, 0.1803, 0.5251,
        0.7435, 0.0000, 0.1322])

In [27]:
expected_state_action_values = (t * 0.99) + reward_batch

In [43]:
expected_state_action_values 

tensor([-5.6240, -9.1518, -5.8075,  1.4538, -1.5731, -9.3266,  6.4609,  3.1165,
         7.0990,  6.0695,  7.7057, -6.1558])

In [31]:
o.shape

torch.Size([12])

In [28]:
from torch.nn import functional as F

loss = F.smooth_l1_loss(o, expected_state_action_values)

/tmp/ipykernel_95835/2370427825.py:3: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([12, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.smooth_l1_loss(o, expected_state_action_values)


In [49]:
loss.backward()

In [32]:
loss = F.smooth_l1_loss(o, expected_state_action_values)